<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 内存高效的模型权重加载


- 本笔记本提供了在 GPU（或 CPU）内存有限的情况下加载更大预训练或微调模型的技巧。
- 具体来说，它针对以下情况：你使用 `torch.save(model.state_dict(), "model.pth")` 保存了模型（例如在第 5-7 章中），并希望在新的会话中加载以继续预训练或进行额外微调。
- 虽然示例使用的是大型语言模型（LLM），但本笔记本讲解的方法是通用的，适用于加载任何 PyTorch 模型，而不仅仅是 LLM。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/memory-efficient-loading/memory-efficient-loading.webp" width="800px">

In [1]:
from importlib.metadata import version

pkgs = [
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

torch version: 2.6.0


&nbsp;
## 1. 基准测试工具


- 首先，我们定义一些工具函数来跟踪 VRAM（GPU 显存）使用情况
- 稍后，我们还将引入一个工具来跟踪主系统内存（CPU 内存）
- 当我们在后续示例中应用这些函数时，它们的用途将变得清晰


In [2]:
import gc
import time
import torch


def start_memory_tracking():
    """Initialize GPU memory tracking."""
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    else:
        print("This notebook is intended for CUDA GPUs but CUDA is not available.")

def print_memory_usage():
    max_gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)  # Convert bytes to GB
    print(f"Maximum GPU memory allocated: {max_gpu_memory:.1f} GB")

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(3)  # some buffer time to allow memory to clear
    torch.cuda.reset_peak_memory_stats()
    max_memory_allocated = torch.cuda.max_memory_allocated(device) / (1024 ** 3)
    print(f"Maximum GPU memory allocated: {max_memory_allocated:.1f} GB")

&nbsp;
## 2. 模型设置


- 本代码部分用于设置模型本身
- 在这里，我们使用“large”版本的 GPT-2 模型以增加示例的复杂度（你也可以使用“gpt2-small（124M）”版本来降低本笔记本的内存需求和运行时间）


In [3]:
from previous_chapters import GPTModel
# If the `previous_chapters.py` file is not available locally,
# you can import it from the `llms-from-scratch` PyPI package.
# For details, see: https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
# E.g.,
# from llms_from_scratch.ch04 import GPTModel



BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-xl (1558M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

- 现在，让我们看看 GPU 内存监控函数的实际效果：


# start_memory_tracking()


model = GPTModel(BASE_CONFIG)
device = torch.device("cuda")
model.to(device)

print_memory_usage()

- 此外，我们可以通过传入一些示例张量来确保模型能够正常运行


In [5]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

- 接下来，假设我们正在对模型进行预训练并保存以便日后使用
- 为了简化，我们在此跳过实际的预训练，仅保存初始化后的模型（但相同的概念同样适用）


In [6]:
# Training code would go here...

model.train()
torch.save(model.state_dict(), "model.pth")

- 最后，我们在 Python 会话中删除模型和示例张量，以重置 GPU 内存


In [7]:
del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 3. 加载权重


- 现在进入有趣的部分：加载预训练模型权重
- 我们先看看加载之前保存的模型需要多少 GPU 内存


In [8]:
# Then load pretrained weights

start_memory_tracking()

model = GPTModel(BASE_CONFIG)
model.to(device)

model.load_state_dict(
    torch.load("model.pth", map_location=device, weights_only=True)
)
model.to(device)
model.eval();

print_memory_usage()

Maximum GPU memory allocated: 12.8 GB


- 注意，此时占用的内存比前一次会话大约多 2 倍
- 这是因为在短时间内，内存中同时存在两个相同的模型：
  - 第一个通过 `model.to(device)` 加载到设备上
  - 第二个通过代码行 `model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))` 加载；最终，加载的模型权重会被复制到主模型中，并丢弃 `state_dict`，但在此过程中短时间内，内存中同时存在主模型和加载的 `state_dict`
- 接下来的章节将着重解决这一问题
- 但首先，我们先测试模型并重置 GPU 内存


In [9]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 4. 顺序加载模型权重


- 针对上一节中提到的模型权重在 GPU 内存中出现两份的问题，一种解决方法是**顺序加载模型**。
- 具体步骤如下：
  - 先将模型初始化并加载到 GPU 内存
  - 再将模型权重加载到 CPU 内存
  - 最后将每个参数逐一拷贝到 GPU 内存中


In [10]:
start_memory_tracking()

model = GPTModel(BASE_CONFIG).to(device)

state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

print_memory_usage()

# Sequentially copy weights to the model's parameters
with torch.no_grad():
    for name, param in model.named_parameters():
        if name in state_dict:
            param.copy_(state_dict[name].to(device))
        else:
            print(f"Warning: {name} not found in state_dict.")

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB


- 如上所示，内存使用量相比之前大幅降低。
- 注意内存从 6.4 GB 增加到 6.7 GB，这是因为最初只有模型本身在内存中，而在加载参数时，我们临时将一个参数张量移到 GPU，以便使用 `".to"` 方法分配给模型。
- 总体来看，这是一种显著的优化。
- 同样地，我们可以先快速测试模型，然后重置 GPU 内存，为下一节做准备。


In [11]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input, state_dict, param
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 5. 在低 CPU 内存环境下加载模型


- 在前一节中，我们通过先将权重 (`state_dict`) 加载到 CPU 内存，再逐个复制到模型中，从而减少了 GPU 内存占用
- 然而，如果 CPU 内存有限，该怎么办呢？
- 本节介绍使用 PyTorch 的 `"meta"` 设备方法，在 GPU 内存充足但 CPU 内存有限的机器上加载模型
- 首先，我们定义一个方便的函数来监控 CPU 内存使用情况


In [12]:
import os
import psutil
from threading import Thread


def memory_usage_in_gb(func, *args, **kwargs):
    process = psutil.Process(os.getpid())

    # Measure the baseline memory usage before running the function
    baseline_mem = process.memory_info().rss / 1024 ** 3  # in GB

    # Start monitoring memory in a separate thread
    mem_usage = []
    done = False

    def monitor_memory():
        while not done:
            mem_usage.append(process.memory_info().rss / 1024 ** 3)  # Convert to GB
            time.sleep(0.1)

    t = Thread(target=monitor_memory)
    t.start()

    # Run the function
    func(*args, **kwargs)

    # Stop monitoring
    done = True
    t.join()

    peak_mem_usage_gb = max(mem_usage) - baseline_mem
    return peak_mem_usage_gb


- 首先，让我们跟踪上一节中顺序加载权重方法的 CPU 内存使用情况


In [13]:
def load_sequentially():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG).to(device)

    state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name].to(device))
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_sequentially)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB
-> Maximum CPU memory allocated: 6.3 GB


- 现在，假设我们有一台 CPU 内存较低但 GPU 内存较大的机器
- 我们可以通过引入 PyTorch 的所谓 "meta" 设备来在 CPU 内存和 GPU 内存使用之间进行权衡
- PyTorch 的 meta 设备是一种特殊的设备类型，它允许你创建张量而不为其数据分配实际内存，有效地创建“meta”张量
- 这对于模型分析或架构定义等任务非常有用，因为你只需要张量的形状和类型，而无需承担内存分配的开销


In [14]:
def load_sequentially_with_meta():
    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    state_dict = torch.load("model.pth", map_location=device, weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name])
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(load_sequentially_with_meta)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 1.3 GB


- 如上所示，通过在 meta 设备上创建模型并将权重直接加载到 GPU 内存中，我们有效地降低了对 CPU 内存的需求
- 可能有人会问：“那么顺序加载权重是否仍然必要？它与原始方法相比如何？”
- 我们来看一下简单的 PyTorch 权重加载方法作为对比（来自本笔记本中第一个权重加载部分）：


In [15]:
def baseline():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG)
    model.to(device)

    model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))
    model.to(device)
    model.eval();

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(baseline)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 4.4 GB


- 如上所示，没有使用 meta 设备的“简单”权重加载会占用更多内存
- 换句话说，如果你的机器 CPU 内存有限，可以使用 meta 设备的方法将模型权重直接加载到 GPU 内存中，从而降低 CPU 内存峰值占用


&nbsp;
## 6. 使用 `mmap=True`（推荐方式）


- 对于中级或高级的 `torch.load` 用户，你可能会好奇这些方法与 PyTorch 中的 `mmap=True` 设置相比如何。  
- PyTorch 中的 `mmap=True` 设置启用了内存映射文件 I/O，这允许张量直接从磁盘存储访问数据，从而减少内存使用量，因为如果 RAM 有限，它不会将整个文件加载到内存中。  
- 另外，可以参考 [mikaylagawarecki](https://github.com/rasbt/LLMs-from-scratch/issues/402) 的有用评论。  
- 初看起来，它可能比前面的顺序加载方法效率略低：


In [37]:
def best_practices():
  with torch.device("meta"):
      model = GPTModel(BASE_CONFIG)

  model.load_state_dict(
      torch.load("model.pth", map_location=device, weights_only=True, mmap=True),
      assign=True
  )

  print_memory_usage()

peak_memory_used = memory_usage_in_gb(best_practices)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 5.9 GB


- CPU RAM 使用量高的原因是该机器上有足够的 CPU 内存可用。  
- 然而，如果你在一台 CPU 内存有限的机器上运行，`mmap` 方法将使用更少的内存。


&nbsp;
## 7. 其他方法


- 本笔记本主要关注 PyTorch 中用于加载权重的简单内置方法
- 对于 CPU 内存有限的情况，推荐的方法是前面讲解的 `mmap=True` 方法
- 另外，还有一种可选方法是暴力处理，即将每个权重张量单独保存和加载：


In [13]:
model = GPTModel(BASE_CONFIG)
# Assume `model` is your trained model
state_dict = model.state_dict()

# Create a directory to store individual parameter files
os.makedirs("model_parameters", exist_ok=True)

# Save each parameter tensor separately
for name, param in state_dict.items():
    torch.save(param.cpu(), f"model_parameters/{name}.pt")

del model

In [16]:
def load_individual_weights():

    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    print_memory_usage()
    param_dir = "model_parameters"

    with torch.no_grad():
        for name, param in model.named_parameters():
            weight_path = os.path.join(param_dir, f"{name}.pt")
            if os.path.exists(weight_path):
                param_data = torch.load(weight_path, map_location="cpu", weights_only=True)
                param.copy_(param_data)
                del param_data  # Free memory
            else:
                print(f"Warning: {name} not found in {param_dir}.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_individual_weights)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 0.3 GB
